# Práctica 5: Clasificador de distancia minima


## calcCentroids() - Fase de aprendizaje
El objetivo de esta función es calcular para cada clase su centroide (el punto/vector medio de todos sus datos).

La función recibe:  
X: una matriz con las características de cada muestra.  
- Cada fila = una muestra.   
- Cada columna = una característica.  
y: un vector con las etiquetas o clases correspondientes

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
def calcCentroids(X, y):
    classes = np.unique(y)
    centroids = [] # Lista vacia donde se guardaran los centroides.

    for c in classes:
        X_c = X[y == c] # Selecciona las muestras de la clase
        mu_c = X_c.mean(axis=0) # Calculo del promedio de las muestras
        centroids.append(mu_c) # Guarda el centroide en la lista

    return classes, np.array(centroids)

`classes = np.unique(y)` devuelve las clases unicas del dataset.

Posteriormente se ejecutara el bucle que corresponde directamente a la fase de aprendizaje:

$$m_k=\frac{1}{N_E(\omega_k)}\sum_{j=1}^{N_E(\omega_k)} x^{j}$$

## predict() - Fase de clasificación
El objetivo de esta función es tomar nuevas muestras X y determinar a qué clase pertenecen, comparando cada muestra con todos los centroides y eligiendo el más cercano.

La función recibe:  
X: matriz con las muestras a clasificar.  
classes: array con los nombres o etiquetas de cada clase.    
centroids: matriz donde cada fila es el centroide de cada clase.  

In [36]:
def predict(X, classes, centroids):
    X = np.asarray(X)
    centroids = np.asarray(centroids)

    distances = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
    minIndex = np.argmin(distances, axis=1) # Distancia minima de cada fila
    return classes[minIndex]


`distances = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)` Esta parte del codigo es el responsable de calcular la distancia euclidiana entre cada muestra y cada centroide.
$$d( \tilde{x}, m_k) = \sqrt{\sum_{i=1}^{n}(\tilde{x}_i-m_{ki})^2}$$
El resultado que arroja es una matriz de tamaño (n_muestras, n_clases) donde en cada posición [i][j] se guarda la distancia de la muestra i al centroide de la clase k.

Posteriormente con `minIndex = np.argmin(distances, axis=1)` se hace la busqueda del indice del centroide más cercano para cada muestra.

## showConfusionMatrix() 
Esta función nos permite comparar ambas y ver cuántas veces tu modelo acertó o se confundio entre clases.  
La función recibe:  
y_true: etiquetas reales  
y_pred: etiquetas que predijo el modelo  
classes: lista con las clases posibles, para ordenar la matriz

In [38]:
def showConfusionMatrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred, labels=classes)

    print("\nConfusion Matrix:")
    print(pd.DataFrame(cm,
                       index=[f"Real {c}" for c in classes],
                       columns=[f"Pred {c}" for c in classes]))

# Función principal
### Carga y limpieza del dataset
def main():
    df = pd.read_csv("Iris.csv")

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    X = X.apply(pd.to_numeric, errors="coerce")
    maskValidRows = ~X.isna().any(axis=1)
    X = X[maskValidRows]
    y = y[maskValidRows]

    X = X.values
    y = y.values

    print("Dataset successfully loaded")
    print("Total size:", X.shape[0], "Samples,", X.shape[1], "Features")


## Hold-Out 70/30
Se aplica el metodo de validación Hold-Out 70/30, donde entrenamos al modelo en el 70% y lo evaluamos una sola vez en el 30% que no vio durante el entrenamiento.

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0, stratify=y
    )

### Normalización
Aplicamos normalización con uso de la función StandardScale.  
`fit_transform` en `X_train` Calcula la media y desviación estandar de cada caracteristica usando solo el conjunto de entrenamiento

`transform` en `X_test` no recalcula media ni sigma, solamente aplica las del entrenamiento. 

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

### Entrenamiento y predicción (usando el clasficador)

    classes, centroids = calcCentroids(X_train, y_train)
    y_pred = predict(X_test, classes, centroids)

### Accuracy (medida de desempeño)
Aqui se calcula la proporción de predicciones correctas y se muestra la matriz de confusión

    acc = accuracy_score(y_test, y_pred)

    print("\n============= 70/30 Hold-Out Results =============")
    print(f"Accuracy: {acc:.4f}")
    showConfusionMatrix(y_test, y_pred, classes)


### 10-Fold Cross Validation
En este bloque de codigo lo que realiza el KFold es dividir nuestro dataset en 10 partes aproximadamente del mismo tamaño.  
En cada iteración, tomamos 1 fold como test y los 9 restantes como entrenamiento. 

   `kf = KFold(n_splits=10, shuffle=True, random_state=0)
    accuracies = []`

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        # Extraemos los datos correspondientes al fold.
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Normalizamos por fold (Ajustado solamente al entrenamiento del fold)
        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X_train)
        X_test_fold = scaler.transform(X_test)

        # Entrenamiento y preddición
        classes, centroids = calcCentroids(X_train_fold, y_train)
        y_pred = predict(X_test_fold, classes, centroids)

        # Calculo del desempeño para este fold
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
        print(f"Fold {fold}: Accuracy = {acc:.4f}")

Para finalizar mostramos el desempeño promedio de los 10 folds y la variación del desempeño entre folds. 

    print("\n============= 10-Fold Cross Validation Results =============")
    print(f"Accuracy average: {np.mean(accuracies):.4f}")
    print(f"Standard deviation: {np.std(accuracies):.4f}")


In [39]:
def main():
    df = pd.read_csv("Iris.csv")

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    X = X.apply(pd.to_numeric, errors="coerce")
    maskValidRows = ~X.isna().any(axis=1)
    X = X[maskValidRows]
    y = y[maskValidRows]

    X = X.values
    y = y.values

    print("Dataset successfully loaded")
    print("Total size:", X.shape[0], "Samples,", X.shape[1], "Features")

    # Hold-Out split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0, stratify=y
    )

    # Scale using only training data (best practice)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    classes, centroids = calcCentroids(X_train, y_train)
    y_pred = predict(X_test, classes, centroids)

    acc = accuracy_score(y_test, y_pred)

    print("\n============= 70/30 Hold-Out Results =============")
    print(f"Accuracy: {acc:.4f}")
    showConfusionMatrix(y_test, y_pred, classes)

    # 10-Fold Cross Validation (simple version: scaling outside per fold)
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    accuracies = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X_train)
        X_test_fold = scaler.transform(X_test)

        classes, centroids = calcCentroids(X_train_fold, y_train)
        y_pred = predict(X_test_fold, classes, centroids)

        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
        print(f"Fold {fold}: Accuracy = {acc:.4f}")

    print("\n============= 10-Fold Cross Validation Results =============")
    print(f"Accuracy average: {np.mean(accuracies):.4f}")
    print(f"Standard deviation: {np.std(accuracies):.4f}")

if __name__ == "__main__":
    main()


Dataset successfully loaded
Total size: 150 Samples, 5 Features

============= 70/30 Hold-Out Results =============
Accuracy: 0.9556

Confusion Matrix:
                      Pred Iris-setosa  Pred Iris-versicolor  \
Real Iris-setosa                    15                     0   
Real Iris-versicolor                 0                    14   
Real Iris-virginica                  0                     1   

                      Pred Iris-virginica  
Real Iris-setosa                        0  
Real Iris-versicolor                    1  
Real Iris-virginica                    14  
Fold 1: Accuracy = 0.9333
Fold 2: Accuracy = 1.0000
Fold 3: Accuracy = 1.0000
Fold 4: Accuracy = 0.8667
Fold 5: Accuracy = 0.9333
Fold 6: Accuracy = 1.0000
Fold 7: Accuracy = 0.9333
Fold 8: Accuracy = 1.0000
Fold 9: Accuracy = 0.9333
Fold 10: Accuracy = 1.0000

============= 10-Fold Cross Validation Results =============
Accuracy average: 0.9600
Standard deviation: 0.0442
